In [1]:
import pandas as pd
import numpy as np
import os
import csv

cwd = os.getcwd()


In [2]:
#!cp /Users/saulgrimaldo1/Downloads/create_training_data.py code/.
!cat dta/training_data_combined.txt | python code/create_training_data.py > dta/training_data_new.txt

^C
Traceback (most recent call last):
  File "code/create_training_data.py", line 48, in <module>
    if verify_sarcastic(row[2]) and message not in sarcastic_messages :
KeyboardInterrupt


In [4]:
!tail -n 100 dta/training_data_new.txt

tail: cannot open 'dta/training_data_new.txt' for reading: No such file or directory


In [7]:
!tail dta/training_data_combined.txt

984164486859907073|Chris_RWS|@Tracker_TD Obvious sarcasm is obvious||Chris_RWS|1453396112|984164139584114690|en|None|None|1
984164486612443141|Kayla155SOS|RT @Ashton5SOS: @stellaluna_s Sarcasm Stella, sarcasm||None|None|None|it|None|None|1
984164502814928896|proudrepublica|@ummmno21 Yeah it appears that inside jokes and sarcasm are taken as racist remarks....I was recently accused being a racist with a sarcasm remark.. I have siblings that are Mexican and Korean smh|[]|ummmno21|798603640613335040|984149821291458561|en|None|None|1
984164507617517569|_Jackson_White|bro I swear its gonna take me a cool year before I love a girl https://t.co/JCO7RYNHDc||None|None|None|en|None|None|1
984164519349014528|_Jackson_White|RT @kikiborrelli: TAURUS ♉️ earth sign 🌏 - STUBBORN - always need to be right- hate when others have a lack of common sense- either rea…||None|None|None|en|None|None|1
984164528656146432|cammyckes|hmm about 87% of this is accurate everything else... absolutely not https://t.co/

In [ ]:
#df_sarcastic = pd.read_csv('merged_data_v1_1.csv', sep = '|')

df = pd.DataFrame()



with open('dta/training_data_combined.txt', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            header= row
            df = pd.DataFrame(columns = row)
        else:
            if len(header) != len(row):
                continue
            else:
                addition = {i:j for i, j in zip(header,row)}
                df = df.append(addition, ignore_index=True )
                


df.tail()        

In [ ]:
for i in np.unique(df['in_reply_to_status_id']):
    if len(i) != 18:
        print (i)

In [ ]:
#### true exmaples : 
### false examples :
import re
status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_storage = []

for j in df[df['in_reply_to_status_id'] != 'None'].index:
    if (((df['in_reply_to_screen_name'][j][0] != '#')) & 
        bool(re.match(status_id_pattern,df['in_reply_to_status_id'][j]))):
        index_storage.append(j)
    else: continue

len(index_storage)

In [ ]:
#want the clean rows that do not have retreats

status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_alt = []

for j in df[df['in_reply_to_status_id'] == 'None'].index:
    if (bool(re.match(status_id_pattern,df['id'][j]))):
        index_alt.append(j)
    else: continue

len(index_alt)


In [ ]:
#subset those data sets that will not have as much trouble

sub_df = df.iloc[index_storage]
sub_df['url']= "https://twitter.com/"+sub_df['in_reply_to_screen_name']+"/status/"+sub_df['in_reply_to_status_id']
#max(index_storage)
sub_df.head()

In [ ]:
sub_df

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import json
def preceding_tweet(URL_):

    ### TODO  URL_ will be a function of the table 
    #URL_ = "https://twitter.com/verobchfinfan/status/976155584448868353"
    page = requests.get(URL_)

    soup = BeautifulSoup(page.content, 'html.parser')

    html = list(soup.children)[2]

    body = list(html.children)[3]


    ###  entity in tweet that gives a full tweet with low confidence in structure

    p1 = list(body.children)[61]

    p1a = list(p1.children)[5]
    p1b=list(list(p1a.children)[3].children)[1]
    set_of_parent = set(list(p1b.children)[1].get_text().split('\n'))
    set_of_parent = list(map(lambda x: x.strip(),set_of_parent))

   
    ###  entity in tweet that gives a partial tweet with high confidence in structure
    p2 = list(body.children)[65]

    p2a= json.loads(p2.__dict__['attrs']['value'])

    p2a['initialState']['title']

    pattern_1 = r'^[A-Za-z0-9 ]* on Twitter: "'
    pattern_2 = '\u2026'

    if re.match(pattern_1, p2a['initialState']['title']):
        start_index = len(re.match(pattern_1, p2a['initialState']['title']).group(0))
    else:
        start_index = None
    
    
    if pattern_2 in p2a['initialState']['title']:
        stop_index = p2a['initialState']['title'].rfind(pattern_2)
    else: 
        stop_index = None

    partial_tweet = p2a['initialState']['title'][start_index:stop_index]
    
    
    #### pull out tweet preceding

    potential_tweet_list = [el for el in set_of_parent if partial_tweet in el]

    len_counter = 0

    for i in potential_tweet_list:
        current_len = len(i)
        if current_len> len_counter:
            preceding_tweet_x = i
        else:
            pass
    if len(potential_tweet_list)==0:
        return partial_tweet
    else:
        return preceding_tweet_x

In [ ]:
storage_pretweet = []

for x in sub_df['url']:
    url = x.strip()
    try:
        storage_pretweet.append(preceding_tweet(url))
    except:
        storage_pretweet.append(None)
    

In [ ]:
storage_pretweet

In [ ]:
sub_df['first_preceding_tweet'] = storage_pretweet

sub_df.head()

In [ ]:
index_alt

In [ ]:
merged_data = pd.concat([df.iloc[index_alt],sub_df])

print(merged_data.shape)

merged_data.tail(10)

In [ ]:
import re
def clean_text(text):
    text = re.sub(r"(\||\'|\"|\n|#sarcasm|#sarcastic)"," ", str(text),flags=re.IGNORECASE)
    return text

In [ ]:
merged_data['first_preceding_tweet_clean'] = merged_data['first_preceding_tweet'].apply(clean_text)


merged_data_clean = merged_data[['text', 'first_preceding_tweet_clean','sarcastic']]

merged_data_clean.to_csv("dta/merged_data_v4.csv", sep = '|', index = False)

In [ ]:
merged_data_clean.head(10)

In [ ]:
!wc -l dta/merged_data_v4.csv